In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

# 반응 시스템 설정: 메탄-산소 반응 시스템
gas = ct.Solution('gri30.yaml')  # GRI-Mech 3.0 반응 메커니즘 사용
gas.TPX = 1200.0, ct.one_atm, 'CH4:1, O2:2, N2:7'  # 초기 온도, 압력, 조성

# 0D 반응기 설정 (정압 조건 및 비등온)
reactor = ct.IdealGasConstPressureReactor(gas, energy='off')  # 정압 반응기, 에너지 방정식 꺼짐

# 반응기 네트워크
sim = ct.ReactorNet([reactor])

# 시뮬레이션 시간 설정
time_end = 0.05  # 총 시뮬레이션 시간 (초)
n_steps = 100  # 계산 단계 수
time = np.linspace(0, time_end, n_steps)  # 시뮬레이션 시간 배열
dt = time_end / n_steps  # 시간 간격

# 사용자 정의 시간에 따른 온도 프로파일 (예: 선형 증가)
temperature_profile = np.linspace(1200, 1800, n_steps)  # 1200K에서 1800K까지 선형 증가

# 결과 저장을 위한 배열
temperature = np.zeros(n_steps)
pressure = np.zeros(n_steps)
mole_fractions = np.zeros((n_steps, len(gas.species_names)))

# 0D 반응기 시뮬레이션 루프 (사용자 정의 온도 적용)
for i in range(n_steps):
    reactor.T = temperature_profile[i]  # 사용자 지정 온도 프로파일 적용
    sim.advance(time[i])  # 현재 시간까지 시뮬레이션 진행
    
    # 결과 저장
    temperature[i] = reactor.T  # 현재 온도
    pressure[i] = reactor.thermo.P  # 현재 압력 (고정된 상태)
    mole_fractions[i, :] = reactor.thermo.X  # 현재 몰분율

# 결과 플로팅
plt.figure(figsize=(10, 6))

# 온도 그래프
plt.subplot(2, 1, 1)
plt.plot(time, temperature_profile, label='Temperature Profile (K)')
plt.xlabel('Time (s)')
plt.ylabel('Temperature (K)')
plt.grid(True)
plt.legend()

# 주요 종의 몰분율 그래프 (예: CH4, O2, CO2, H2O)
plt.subplot(2, 1, 2)
plt.plot(time, mole_fractions[:, gas.species_index('CH4')], label='CH4')
plt.plot(time, mole_fractions[:, gas.species_index('O2')], label='O2')
plt.plot(time, mole_fractions[:, gas.species_index('CO2')], label='CO2')
plt.plot(time, mole_fractions[:, gas.species_index('H2O')], label='H2O')
plt.xlabel('Time (s)')
plt.ylabel('Mole Fraction')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


AttributeError: attribute 'T' of 'cantera.reactor.ReactorBase' objects is not writable